In [ ]:
!pip install ipycanvas

In [ ]:
import copy

import numpy as np

%matplotlib widget
import matplotlib.pyplot as plt

from scwidgets import (CodeDemo, ParametersBox, CodeChecker, PyplotOutput, ClearedOutput, AnimationOutput)
from widget_code_input import WidgetCodeInput

In [ ]:
#Base example CodeDemo, ignore
from ipywidgets import HBox, VBox, Layout, HTML
from ipywidgets import Layout
import ipywidgets as widgets
from IPython.display import display, HTML

base_code_input = WidgetCodeInput(
   function_name = "my_function",
   function_parameters = "x",
   docstring="""
   Input the docstring here.
   """,
   function_body="# A simple function \nreturn x",
   code_theme = 'nord',
)

code_checker = CodeChecker({
       })
parameters_box = ParametersBox(n = (1, 0, 5, 1, r'$n$'))

fig,ax = plt.subplots()
output = PyplotOutput(fig)
def update_visualizer(n,code_input,visualizers):
    clear_output = visualizers[0]
    plot = visualizers[1]
    x = np.linspace(0, 2, 1000)
    y = code_input.get_function_object()(x)
    ax.scatter(x, y, color='black')
    
base_demo = CodeDemo(
        code_input=base_code_input,
        input_parameters_box=parameters_box,
        visualizers=[ClearedOutput(), output],
        update_visualizers=update_visualizer,
        code_checker=code_checker,
        separate_check_and_update_buttons=True,
    )

In [ ]:

display(base_demo)

# Solution 1: access HTML directly with methods from DOMWidgets interface

Advantage: Relatively simple and allows dev to directly apply stateful behaviour to stateless ipywidgets base widgets 

Drawback: Not intended use of ipywidgets, can become deprecated in future (ipywidgets #3431 -> "DOM structure is an implementation detail that can change")

In [ ]:
#1 Pattern 1: Cue above code_input



css = HTML("""<style>

.stale{
    border : 5px solid;
    border-color: #CC0000;
    height: auto;
}
.refreshed{
    border : 5px solid;
    border-color: #77AC30;
    height: auto;
}
.checked{
    border : 5px solid;
    border-color: #2E2EFF;
    height: auto;
}
</style>
""")

display(css)

box = widgets.HBox([],layout=Layout(width = 'auto'))
box2 = widgets.HBox([],layout=Layout(width = 'auto'))
box.add_class("refreshed")
box2.add_class("checked")

code_input = WidgetCodeInput(
   function_name = "my_function",
   function_parameters = "x",
   docstring="""
   Input the docstring here.
   """,
   function_body="# A simple function \nreturn x",
   code_theme = 'nord',
)

demo = CodeDemo(
        code_input=code_input,
        input_parameters_box=parameters_box,
        visualizers=[ClearedOutput(), output],
        update_visualizers=update_visualizer,
        code_checker=code_checker,
        separate_check_and_update_buttons=True,
    )



display(VBox([box,box2,demo]))


def wear(change):
        box.remove_class("refreshed")
        box.add_class("stale") 
def refresh(change):
        box.remove_class("stale")
        box.add_class("refreshed")



demo.code_input.observe(wear,'function_body')
demo.input_parameters_box.observe(wear,"value")
demo.update_button.on_click(refresh)





In [ ]:
# Pattern 2: Cue at right of CodeDemo from top of WidgetCodeInput to bottom of last visualizer

box = widgets.HBox([],layout=Layout(width = 'auto'))
box2 = widgets.HBox([],layout=Layout(width = 'auto'))
box.add_class("refreshed")
box2.add_class("checked")

code_input2 = WidgetCodeInput(
   function_name = "my_function",
   function_parameters = "x",
   docstring="""
   Input the docstring here.
   """,
   function_body="# A simple function \nreturn x",
   code_theme = 'nord',
)
parameters_box2 = ParametersBox(n = (1, 0, 5, 1, r'$n$'))


demo2 = CodeDemo(
        code_input=code_input2,
        input_parameters_box=parameters_box2,
        visualizers=[ClearedOutput(), output],
        update_visualizers=update_visualizer,
        code_checker=code_checker,
        separate_check_and_update_buttons=True,
    )
demo2.layout=Layout(width = '100%')
display(HBox([demo2,box,box2]))


def wear(change):
        box.remove_class("refreshed")
        box.add_class("stale") 
def refresh(change):
        box.remove_class("stale")
        box.add_class("refreshed")



demo2.code_input.observe(wear,'function_body')
demo2.input_parameters_box.observe(wear,"value")
demo2.update_button.on_click(refresh)




In [ ]:
# Pattern 3 (not implemented): Cue at the right of WidgetCodeInput without 
# descending to visualizers
# Must change the logic, since CodeDemo is a VBox, must allow for code_input to be a HBox)


# Solution 2: Create own custom Widget 

Advantage: Maybe easier to LTS? I do not know enough JS to know...

Drawback: requires implementing both backend and frontend (https://ipywidgets.readthedocs.io/en/7.x/examples/Widget%20Custom.html)

# Solution 3: Use ipycanvas

Advantage: an already implemented and apparently live Custom Widget that allows more liberty in creating canvas/stateful cues

Drawback: not sure how the interaction with frontend works and not sure of implications to different platforms (Noto, editors, browsers, etc..)

In [ ]:
## must pip install ipycanvas for next cells 

## https://ipycanvas.readthedocs.io/en/latest/



from ipycanvas import Canvas

# Pattern 4: same as Pattern 1 but with ipycanvas
update_cue = Canvas(width=10,height=10)
update_cue.layout=Layout(width='auto',height='auto')
update_cue.fill_style = "#77AC30"
update_cue.fill_rect(0,0,100)

check_cue = Canvas(width=10,height=10)
check_cue.layout=Layout(width='auto',height='auto')
check_cue.fill_style = "#2E2EFF"
check_cue.fill_rect(0,0,100)



code_input3 = WidgetCodeInput(
   function_name = "my_function",
   function_parameters = "x",
   docstring="""
   Input the docstring here.
   """,
   function_body="# A simple function \nreturn x",
   code_theme = 'nord',
)
parameters_box3 = ParametersBox(n = (1, 0, 5, 1, r'$n$'))
demo3 = CodeDemo(
        code_input=code_input3,
        input_parameters_box=parameters_box3,
        visualizers=[ClearedOutput(), output],
        update_visualizers=update_visualizer,
        code_checker=code_checker,
        separate_check_and_update_buttons=True,
    )


display(VBox([update_cue,check_cue,demo3]))


def wear(change):
        update_cue.fill_style = "#CC0000"
        update_cue.fill_rect(0,0,100)
def refresh(change):
        update_cue.fill_style = "#77AC30"
        update_cue.fill_rect(0,0,100)  



demo3.code_input.observe(wear,'function_body')
demo3.input_parameters_box.observe(wear,"value")
demo3.update_button.on_click(refresh)


In [ ]:
# Pattern 5 : circle cues 

update_cue = Canvas(width=80,height=80)
update_cue.layout=Layout(width='30px',height='auto')

update_cue.fill_style = "#77AC30"
update_cue.fill_circle(40,40,20)


check_cue = Canvas(width=80,height=80)
check_cue.layout=Layout(width='30px',height='auto')
check_cue.fill_style = "#2E2EFF"
check_cue.fill_circle(40,40,20)

cues = VBox([update_cue,check_cue])


code_input4 = WidgetCodeInput(
   function_name = "my_function",
   function_parameters = "x",
   docstring="""
   Input the docstring here.
   """,
   function_body="# A simple function \nreturn x",
   code_theme = 'nord',
)
parameters_box4 = ParametersBox(n = (1, 0, 5, 1, r'$n$'))


demo4 = CodeDemo(
        code_input=code_input4,
        input_parameters_box=parameters_box4,
        visualizers=[ClearedOutput(), output],
        update_visualizers=update_visualizer,
        code_checker=code_checker,
        separate_check_and_update_buttons=True,
    )


display(HBox([demo4,cues]))


def wear(change):
        update_cue.fill_style = "#CC0000"
        update_cue.fill_circle(40,40,20)

def refresh(change):
        update_cue.fill_style = "#77AC30"
        update_cue.fill_circle(40,40,20)
        


demo4.code_input.observe(wear,'function_body')
demo4.input_parameters_box.observe(wear,"value")
demo4.update_button.on_click(refresh)


